### **0. Set-up**

In [1]:
# Import libraries and utils
%run '../../utils.ipynb'

In [2]:
# Get api key
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Set client
client = OpenAI()

In [3]:
# Load dataframe
cleaned_nl_simlex = pd.read_csv("../../../data/dataset/cleaned-nl-simlex-999.csv")

# Select subset
# cleaned_nl_simlex = cleaned_nl_simlex.head(150)

# Convert to tuple
tuples_list = list(zip(cleaned_nl_simlex['word1'], cleaned_nl_simlex['word2']))

In [5]:
# Show results
tuples_list

[('oud', 'nieuw'),
 ('slim', 'intelligent'),
 ('hard', 'moeilijk'),
 ('gelukkig', 'vrolijk'),
 ('hard', 'stoer'),
 ('snel', 'razendsnel'),
 ('gelukkig', 'blij'),
 ('kort', 'lang'),
 ('dom', 'stom'),
 ('vreemd', 'eigenaardig'),
 ('breed', 'smal'),
 ('makkelijk', 'moeilijk'),
 ('moeilijk', 'gemakkelijk'),
 ('slim', 'dom'),
 ('krankzinnig', 'gek'),
 ('gelukkig', 'kwaad'),
 ('uitgebreid', 'groot'),
 ('moeilijk', 'simpel'),
 ('nieuw', 'vers'),
 ('scherp', 'saai'),
 ('vlug', 'snel'),
 ('dom', 'dwaas'),
 ('prachtig', 'fantastisch'),
 ('eigenaardig', 'vreemd'),
 ('gelukkig', 'boos'),
 ('smal', 'breed'),
 ('eenvoudig', 'gemakkelijk'),
 ('oud', 'vers'),
 ('kennelijk', 'duidelijk'),
 ('betaalbaar', 'goedkoop'),
 ('leuk', 'grootmoedig'),
 ('raar', 'vreemd'),
 ('vreemd', 'normaal'),
 ('slecht', 'immoreel'),
 ('verdrietig', 'grappig'),
 ('prachtig', 'geweldig'),
 ('schuldig', 'beschaamd'),
 ('mooi', 'prachtig'),
 ('zelfverzekerd', 'zeker'),
 ('dom', 'onderontwikkeld'),
 ('groot', 'flexibel'),
 ('aar

### **1. Define and Evaluate Parameters**

In [36]:
# Define prompt
prompt = ("Classificeer de semantische gelijkenis van elk woordpaar in de hiërarchische categorieën: "
          "'zeer ongelijk', 'ongelijk', 'neutraal', 'gelijk' en 'zeer gelijk'. "
          "Het antwoord moet strikt voldoen aan de structuur: "
          "[('woord1', 'woord2', <classificatie>), ('woord3', 'woord4', <classificatie>), ...] "
          " Geef geen extra uitleg of context.")

In [37]:
# Define model
model = "gpt-3.5-turbo-0125"

# Set sample size
sample_size = 15

# Delay between individual API calls
delay = 15.0

# Define number of sublists
n_sublists = 30

In [38]:
# Split list
chunks = split_into_n_lists(tuples_list, n_sublists)

# Count number of lists
print(len(chunks))

30


In [41]:
# Print prompts for each chunk
print_prompts(chunks, prompt)

Classificeer de semantische gelijkenis van elk woordpaar in de hiërarchische categorieën: 'zeer ongelijk', 'ongelijk', 'neutraal', 'gelijk' en 'zeer gelijk'. Het antwoord moet strikt voldoen aan de structuur: [('woord1', 'woord2', <classificatie>), ('woord3', 'woord4', <classificatie>), ...]  Geef geen extra uitleg of context. --- ('oud', 'nieuw'), ('slim', 'intelligent'), ('hard', 'moeilijk'), ('gelukkig', 'vrolijk'), ('hard', 'stoer'), ('snel', 'razendsnel'), ('gelukkig', 'blij'), ('kort', 'lang'), ('dom', 'stom'), ('vreemd', 'eigenaardig'), ('breed', 'smal'), ('makkelijk', 'moeilijk'), ('moeilijk', 'gemakkelijk'), ('slim', 'dom'), ('krankzinnig', 'gek'), ('gelukkig', 'kwaad'), ('uitgebreid', 'groot'), ('moeilijk', 'simpel'), ('nieuw', 'vers'), ('scherp', 'saai'), ('vlug', 'snel'), ('dom', 'dwaas'), ('prachtig', 'fantastisch'), ('eigenaardig', 'vreemd'), ('gelukkig', 'boos'), ('smal', 'breed'), ('eenvoudig', 'gemakkelijk'), ('oud', 'vers'), ('kennelijk', 'duidelijk'), ('betaalbaar', 

In [42]:
# Load encoding
encoding = tiktoken.get_encoding("cl100k_base")

# Count tokens per chunk
token_counts = count_tokens_with_tiktoken(chunks, prompt)

# Show results
print("Token counts for each formatted prompt:", token_counts)

Token counts for each formatted prompt: [420, 430, 410, 405, 379, 392, 395, 382, 382, 382, 402, 385, 390, 378, 397, 410, 394, 416, 381, 382, 386, 421, 391, 411, 415, 415, 436, 433, 420, 433]


### **2. Extract and Process Data**

In [43]:
# Get results from API
# response = get_responses(chunks, prompt, model, sample_size, delay)

Processing: 100%|██████████| 450/450 [2:49:56<00:00, 22.66s/chunk]  

Total time taken: 10196.75 seconds


In [44]:
# Define filepath
file_path = '../../../data/response/nl/gpt-3.5-turbo-0125/f5.json'

# Check if file already exists
if not os.path.exists(file_path):
    with open(file_path, 'w') as f:
        json.dump(response, f)
        print("File saved successfully.")
else:
    print("File already exists. JSON was not saved to prevent overwriting.")

File saved successfully.


In [59]:
# Process data into dictionary
data_dict = process_responses_categorical(response)

# Check for values higher/lower then sample size
higher_lower_samples = {key: value for key, value in data_dict.items() if len(value) < sample_size or len(value) > sample_size}

# Show results
print(higher_lower_samples)

{('wolk', 'nevel'): ['zeer gelijk', 'ongelijk', 'ongelijk', 'ongelijk', 'gelijk', 'ongelijk', 'gelijk', 'ongelijk', 'ongelijk', 'gelijk', 'ongelijk', 'gelijk', 'gelijk', 'ongelijk'], ('worden', 'lijken'): ['gelijk', 'gelijk', 'ongelijk', 'gelijk', 'ongelijk', 'gelijk', 'ongelijk', 'zeer ongelijk', 'ongelijk', 'gelijk', 'ongelijk', 'ongelijk', 'ongelijk', 'neutraal']}


In [60]:
# Print duplicate word pairs
print_duplicate_word_pairs(cleaned_nl_simlex, data_dict)

Empty DataFrame
Columns: [Combined_Columns]
Index: []
Empty DataFrame
Columns: [Combined_Columns]
Index: []


In [61]:
# Convert dict to Pandas DataFrame
df = create_dataframe(data_dict)

# Show results
df

,word1,word2,similarity_score_1,similarity_score_2,similarity_score_3,similarity_score_4,similarity_score_5,similarity_score_6,similarity_score_7,similarity_score_8,similarity_score_9,similarity_score_10,similarity_score_11,similarity_score_12,similarity_score_13,similarity_score_14,similarity_score_15
0,oud,nieuw,zeer ongelijk,zeer ongelijk,zeer ongelijk,zeer ongelijk,zeer ongelijk,zeer ongelijk,zeer ongelijk,zeer ongelijk,zeer ongelijk,zeer ongelijk,zeer ongelijk,zeer ongelijk,zeer ongelijk,zeer ongelijk,zeer ongelijk
1,slim,intelligent,gelijk,gelijk,gelijk,gelijk,gelijk,gelijk,gelijk,gelijk,gelijk,gelijk,gelijk,gelijk,gelijk,gelijk,gelijk
2,hard,moeilijk,neutraal,ongelijk,neutraal,ongelijk,ongelijk,ongelijk,ongelijk,ongelijk,ongelijk,ongelijk,gelijk,neutraal,ongelijk,ongelijk,ongelijk
3,gelukkig,vrolijk,gelijk,gelijk,ongelijk,gelijk,ongelijk,ongelijk,neutraal,ongelijk,gelijk,gelijk,neutraal,gelijk,neutraal,gelijk,gelijk
4,hard,stoer,neutraal,gelijk,ongelijk,ongelijk,ongelijk,gelijk,zeer ongelijk,ongelijk,ongelijk,neutraal,ongelijk,ongelijk,ongelijk,ongelijk,neutraal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992,samenvoegen,verwerven,ongelijk,ongelijk,ongelijk,ongelijk,ongelijk,ongelijk,ongelijk,ongelijk,ongelijk,ongelijk,ongelijk,ongelijk,ongelijk,ongelijk,ongelijk
993,sturen,bijwonen,zeer ongelijk,ongelijk,ongelijk,ongelijk,neutraal,ongelijk,ongelijk,ongelijk,ongelijk,ongelijk,ongelijk,ongelijk,ongelijk,ongelijk,zeer ongelijk
994,verzamelen,bijwonen,gelijk,neutraal,gelijk,ongelijk,gelijk,ongelijk,ongelijk,ongelijk,gelijk,ongelijk,gelijk,zeer gelijk,ongelijk,ongelijk,ongelijk
995,opnemen,intrekken,ongelijk,ongelijk,ongelijk,ongelijk,gelijk,ongelijk,ongelijk,ongelijk,zeer ongelijk,ongelijk,gelijk,ongelijk,ongelijk,ongelijk,ongelijk


In [62]:
# Count null values
count_null_values = df.isnull().sum()

# Show results
print("Null value counts per column:", count_null_values)

Null value counts per column: word1                  0
word2                  0
similarity_score_1     0
similarity_score_2     0
similarity_score_3     0
similarity_score_4     0
similarity_score_5     0
similarity_score_6     0
similarity_score_7     0
similarity_score_8     0
similarity_score_9     0
similarity_score_10    0
similarity_score_11    0
similarity_score_12    0
similarity_score_13    0
similarity_score_14    0
similarity_score_15    2
dtype: int64


In [63]:
# Check for rows with at least one null value
rows_with_null = df[df.isnull().any(axis=1)]

# Show results
rows_with_null

,word1,word2,similarity_score_1,similarity_score_2,similarity_score_3,similarity_score_4,similarity_score_5,similarity_score_6,similarity_score_7,similarity_score_8,similarity_score_9,similarity_score_10,similarity_score_11,similarity_score_12,similarity_score_13,similarity_score_14,similarity_score_15
288,wolk,nevel,zeer gelijk,ongelijk,ongelijk,ongelijk,gelijk,ongelijk,gelijk,ongelijk,ongelijk,gelijk,ongelijk,gelijk,gelijk,ongelijk,None
952,worden,lijken,gelijk,gelijk,ongelijk,gelijk,ongelijk,gelijk,ongelijk,zeer ongelijk,ongelijk,gelijk,ongelijk,ongelijk,ongelijk,neutraal,None


In [50]:
# Show results
response

["[('oud', 'nieuw', zeer ongelijk'), ('slim', 'intelligent', gelijk'), ('hard', 'moeilijk', neutraal'), ('gelukkig', 'vrolijk', gelijk'), ('hard', 'stoer', neutraal'), ('snel', 'razendsnel', gelijk'), ('gelukkig', 'blij', gelijk'), ('kort', 'lang', zeer ongelijk'), ('dom', 'stom', gelijk'), ('vreemd', 'eigenaardig', gelijk'), ('breed', 'smal', zeer ongelijk'), ('makkelijk', 'moeilijk', zeer ongelijk'), ('moeilijk', 'gemakkelijk', zeer ongelijk'), ('slim', 'dom', zeer ongelijk'), ('krankzinnig', 'gek', gelijk'), ('gelukkig', 'kwaad', zeer ongelijk'), ('uitgebreid', 'groot', gelijk'), ('moeilijk', 'simpel', neutraal'), ('nieuw', 'vers', gelijk'), ('scherp', 'saai', zeer ongelijk'), ('vlug', 'snel', gelijk'), ('dom', 'dwaas', gelijk'), ('prachtig', 'fantastisch', gelijk'), ('eigenaardig', 'vreemd', gelijk'), ('gelukkig', 'boos', zeer ongelijk'), ('smal', 'breed', zeer ongelijk'), ('eenvoudig', 'gemakkelijk', gelijk'), ('oud', 'vers', zeer ongelijk'), ('kennelijk', 'duidelijk', neutraal'),

In [64]:
# Extract missing word pairs
missing_word_pair_list = list(zip(rows_with_null['word1'], rows_with_null['word2']))

# Show results
missing_word_pair_list

[('wolk', 'nevel'), ('worden', 'lijken')]

In [65]:
# Format message
formatted_prompt = format_prompt(missing_word_pair_list, prompt)
messages = [{"role": "user", "content": formatted_prompt}]

# Make API call
# completion = client.chat.completions.create(
#     model=model,
#     messages=messages,
#     n=1)

# Store the response content
missing_word_pairs_response = [completion.choices[0].message.content]

In [66]:
# Show results
missing_word_pairs_response

["[('wolk', 'nevel', ongelijk), ('worden', 'lijken', neutraal)]"]

In [67]:
# Manually fix inconsistencies
df.loc[(df['word1'] == 'wolk') & (df['word2'] == 'nevel'), 'similarity_score_15'] = 'ongelijk'

# Show results
df.loc[(df['word1'] == 'wolk') & (df['word2'] == 'nevel')]

,word1,word2,similarity_score_1,similarity_score_2,similarity_score_3,similarity_score_4,similarity_score_5,similarity_score_6,similarity_score_7,similarity_score_8,similarity_score_9,similarity_score_10,similarity_score_11,similarity_score_12,similarity_score_13,similarity_score_14,similarity_score_15
288,wolk,nevel,zeer gelijk,ongelijk,ongelijk,ongelijk,gelijk,ongelijk,gelijk,ongelijk,ongelijk,gelijk,ongelijk,gelijk,gelijk,ongelijk,ongelijk


In [68]:
# Manually fix inconsistencies
df.loc[(df['word1'] == 'worden') & (df['word2'] == 'lijken'), 'similarity_score_15'] = 'neutraal'

# Show results
df.loc[(df['word1'] == 'worden') & (df['word2'] == 'lijken')]

,word1,word2,similarity_score_1,similarity_score_2,similarity_score_3,similarity_score_4,similarity_score_5,similarity_score_6,similarity_score_7,similarity_score_8,similarity_score_9,similarity_score_10,similarity_score_11,similarity_score_12,similarity_score_13,similarity_score_14,similarity_score_15
952,worden,lijken,gelijk,gelijk,ongelijk,gelijk,ongelijk,gelijk,ongelijk,zeer ongelijk,ongelijk,gelijk,ongelijk,ongelijk,ongelijk,neutraal,neutraal


In [69]:
# Check for rows with at least one null value
rows_with_null = df[df.isnull().any(axis=1)]

# Show results
rows_with_null

,word1,word2,similarity_score_1,similarity_score_2,similarity_score_3,similarity_score_4,similarity_score_5,similarity_score_6,similarity_score_7,similarity_score_8,similarity_score_9,similarity_score_10,similarity_score_11,similarity_score_12,similarity_score_13,similarity_score_14,similarity_score_15


In [70]:
# Show results
df

,word1,word2,similarity_score_1,similarity_score_2,similarity_score_3,similarity_score_4,similarity_score_5,similarity_score_6,similarity_score_7,similarity_score_8,similarity_score_9,similarity_score_10,similarity_score_11,similarity_score_12,similarity_score_13,similarity_score_14,similarity_score_15
0,oud,nieuw,zeer ongelijk,zeer ongelijk,zeer ongelijk,zeer ongelijk,zeer ongelijk,zeer ongelijk,zeer ongelijk,zeer ongelijk,zeer ongelijk,zeer ongelijk,zeer ongelijk,zeer ongelijk,zeer ongelijk,zeer ongelijk,zeer ongelijk
1,slim,intelligent,gelijk,gelijk,gelijk,gelijk,gelijk,gelijk,gelijk,gelijk,gelijk,gelijk,gelijk,gelijk,gelijk,gelijk,gelijk
2,hard,moeilijk,neutraal,ongelijk,neutraal,ongelijk,ongelijk,ongelijk,ongelijk,ongelijk,ongelijk,ongelijk,gelijk,neutraal,ongelijk,ongelijk,ongelijk
3,gelukkig,vrolijk,gelijk,gelijk,ongelijk,gelijk,ongelijk,ongelijk,neutraal,ongelijk,gelijk,gelijk,neutraal,gelijk,neutraal,gelijk,gelijk
4,hard,stoer,neutraal,gelijk,ongelijk,ongelijk,ongelijk,gelijk,zeer ongelijk,ongelijk,ongelijk,neutraal,ongelijk,ongelijk,ongelijk,ongelijk,neutraal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992,samenvoegen,verwerven,ongelijk,ongelijk,ongelijk,ongelijk,ongelijk,ongelijk,ongelijk,ongelijk,ongelijk,ongelijk,ongelijk,ongelijk,ongelijk,ongelijk,ongelijk
993,sturen,bijwonen,zeer ongelijk,ongelijk,ongelijk,ongelijk,neutraal,ongelijk,ongelijk,ongelijk,ongelijk,ongelijk,ongelijk,ongelijk,ongelijk,ongelijk,zeer ongelijk
994,verzamelen,bijwonen,gelijk,neutraal,gelijk,ongelijk,gelijk,ongelijk,ongelijk,ongelijk,gelijk,ongelijk,gelijk,zeer gelijk,ongelijk,ongelijk,ongelijk
995,opnemen,intrekken,ongelijk,ongelijk,ongelijk,ongelijk,gelijk,ongelijk,ongelijk,ongelijk,zeer ongelijk,ongelijk,gelijk,ongelijk,ongelijk,ongelijk,ongelijk


In [72]:
# Define file_path
file_path = '../../../data/prompt/nl/gpt-3.5-turbo-0125/f5.csv'

# Check if file already exists
if not os.path.exists(file_path):
    df.to_csv(file_path, index=False)
    print("File saved successfully.")
else:
    print("File already exists. Dataframe was not saved to prevent overwriting.")

File saved successfully.
